In [45]:
import requests
import pymysql
import json
import pandas as pd
import datetime as dt

### DB 사용 예시

In [44]:
con = pymysql.connect(host='localhost', user='ssafy', password='ssafy', 
                      db='access_db', charset='utf8mb4', autocommit=True)
cur = con.cursor()

sql = "INSERT INTO table_name (value1, value2) VALUES (123, '이름');"
cur.execute(sql)

con.commit()
con.close()


### 환율

In [7]:
def exchange_rate_api(date):
  API_HOST = "https://www.koreaexim.go.kr/site/program/financial/exchangeJSON"
  key = "?authkey=b60MyMtLUDVQyyiDjZRdFGbKxoJBypkl"
  search = "&searchdate="+date
  data = "&data=AP01"
  
  url = API_HOST+key+search+data
  headers = {"Content-Type" : "application/json", "charset" : "UTF-8", "Accept":"*/*"}

  try:
    response = requests.get(url, headers=headers)
    return response
  except Exception as ex:
    print(ex)
  

In [57]:
date_range = pd.date_range(start='1/1/2011', end='12/31/2022')

result_list = []
for date in date_range:
  if dt.date.weekday(date) in (5, 6) :
    continue
  str_date = str(date).split(" ")[0]
  
  r = json.loads(exchange_rate_api(str_date).text)
  
  flag = True
  for data in r:
    if data["result"] == 4:
      flag = False
      break
    result_list.append("{}, {}, {}".format(str_date, d["cur_nm"], d["deal_bas_r"]))
  if flag == False:
    break

In [58]:
len(result_list)

29047

In [59]:
import pickle

In [60]:
with open('data.p', 'wb') as f :
  pickle.dump(result_list, f)

In [55]:
print(result_list[0])

for d in result_list :
  if d['cur_nm'] == "아랍에미리트 디르함":
    print("{}, {}, {}".format("2011-01-01", d["cur_nm"], d["deal_bas_r"]))

{'result': 1, 'cur_unit': 'AED', 'ttb': '306.97', 'tts': '313.17', 'deal_bas_r': '310.07', 'bkpr': '310', 'yy_efee_r': None, 'ten_dd_efee_r': None, 'kftc_bkpr': '310', 'kftc_deal_bas_r': '310.07', 'cur_nm': '아랍에미리트 디르함'}
310.07


In [20]:
r = 
type(r), r

(list,
 [{'result': 1,
   'cur_unit': 'AED',
   'ttb': '306.97',
   'tts': '313.17',
   'deal_bas_r': '310.07',
   'bkpr': '310',
   'yy_efee_r': None,
   'ten_dd_efee_r': None,
   'kftc_bkpr': '310',
   'kftc_deal_bas_r': '310.07',
   'cur_nm': '아랍에미리트 디르함'},
  {'result': 1,
   'cur_unit': 'ATS',
   'ttb': None,
   'tts': None,
   'deal_bas_r': '110.4',
   'bkpr': None,
   'yy_efee_r': None,
   'ten_dd_efee_r': None,
   'kftc_bkpr': None,
   'kftc_deal_bas_r': '110.4',
   'cur_nm': '오스트리아 실링'},
  {'result': 1,
   'cur_unit': 'AUD',
   'ttb': '1,151.81',
   'tts': '1,175.07',
   'deal_bas_r': '1,163.44',
   'bkpr': '1,163',
   'yy_efee_r': None,
   'ten_dd_efee_r': None,
   'kftc_bkpr': '1,163',
   'kftc_deal_bas_r': '1,163.44',
   'cur_nm': '호주 달러'},
  {'result': 1,
   'cur_unit': 'BEF',
   'ttb': None,
   'tts': None,
   'deal_bas_r': '37.66',
   'bkpr': None,
   'yy_efee_r': None,
   'ten_dd_efee_r': None,
   'kftc_bkpr': None,
   'kftc_deal_bas_r': '37.66',
   'cur_nm': '벨기에 프랑'},
